## Full Board Generator

I have programmed the builder in an array, as this was more intuitive. It is later transformed in a flat format.

In [466]:
import random
import Solver_experiment

def generate_full_board(seed=None):
    board = [[0] * 9 for _ in range(9)]
    fill_board(board, seed)
    return flatten_board(board)

def fill_board(board, seed=None):
    if seed is not None:
        random.seed(seed)
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                nums = list(range(1, 10))
                random.shuffle(nums)  # Shuffle the numbers randomly to get different results
                for num in nums:
                    if is_valid(board, i, j, num):
                        board[i][j] = num
                        if fill_board(board, seed):
                            return True
                        board[i][j] = 0
                return False
    return True

def is_valid(board, row, col, num):
    for i in range(9):
        if board[row][i] == num or board[i][col] == num or \
           board[3 * (row // 3) + i // 3][3 * (col // 3) + i % 3] == num:
            return False
    return True

def flatten_board(board):
    flat_board = []
    for row in board:
        flat_board.extend(row)
    return flat_board

def set_random_seed(seed=None):
    if seed is None:
        seed = random.randint(0, 1000)
    random.seed(seed)
    return seed

def generate_multiple_full_boards(num_boards, seed=None):
    set_random_seed(seed)
    full_boards = []
    for i in range(num_boards):
        board_seed = set_random_seed()  # Set a new random seed for each board
        full_boards.append(generate_full_board(board_seed))
    return full_boards

def print_board(board):
    n = int(len(board) ** 0.5)  # Assumes the board is a square (e.g., 9x9)
    for i in range(n):
        if i % 3 == 0 and i != 0:
            print("-" * 21)
        for j in range(n):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")
            # Compute the index for the flat list
            index = i * n + j
            print(board[index], end=" ")
        print()
        
def convert_to_2d_list(flat_board):
    return [flat_board[i*9:(i+1)*9] for i in range(9)]


# Example usage:
seed=7
num_boards = 2
full_boards = generate_multiple_full_boards(num_boards, seed)
for i, board in enumerate(full_boards):
    print(f"Board {i}:")
    print_board(full_boards[i])
   

Board 0:
5 8 7 | 4 1 9 | 2 3 6 
4 1 9 | 2 3 6 | 5 8 7 
2 3 6 | 5 8 7 | 4 1 9 
---------------------
8 5 4 | 7 9 1 | 3 6 2 
7 9 1 | 3 6 2 | 8 5 4 
3 6 2 | 8 5 4 | 7 9 1 
---------------------
1 7 5 | 9 4 8 | 6 2 3 
9 4 8 | 6 2 3 | 1 7 5 
6 2 3 | 1 7 5 | 9 4 8 
Board 1:
3 7 9 | 1 8 6 | 2 5 4 
1 8 6 | 2 5 4 | 3 7 9 
2 5 4 | 3 7 9 | 1 8 6 
---------------------
7 3 1 | 9 6 8 | 5 4 2 
9 6 8 | 5 4 2 | 7 3 1 
5 4 2 | 7 3 1 | 9 6 8 
---------------------
8 9 3 | 6 1 7 | 4 2 5 
6 1 7 | 4 2 5 | 8 9 3 
4 2 5 | 8 9 3 | 6 1 7 


## Game_Board_Generator

In [467]:
from copy import deepcopy

def create_game_board(full_board, num_cells_empty, seed=None):
    if num_cells_empty > 63:
        raise ValueError("Number of cells left must be at least 17")

    game_board = deepcopy(full_board)
    cells = list(range(81))
    random.seed(seed)

    while num_cells_empty > 0 and cells:
        cell_index = random.choice(cells)
        cells.remove(cell_index)
        row, col = cell_index // 9, cell_index % 9
        game_board[row * 9 + col] = '.'

        num_cells_empty -= 1
    
    return game_board

# Example usage:
full_board = full_boards[0]  # Use one of the full Sudoku boards generated earlier
num_cells_empty = 62  # Specify the number of cells to keep filled
seed = 123  # Specify the random seed
game_board = create_game_board(full_board, num_cells_empty, seed)
print("Sudoku Game:")

print_board(game_board)

Sudoku Game:
. . . | . . . | . . . 
. 1 . | . 3 . | . . . 
2 3 6 | . . . | 4 . 9 
---------------------
8 5 . | 7 9 . | . . . 
7 . . | . . . | . 5 . 
. . . | . 5 . | . . . 
---------------------
. . . | . . . | . 2 . 
9 . 8 | . . . | 1 . . 
. . . | . . 5 | . . . 


## Sudoku Game Generator with Difficulty Levels

The below uses the function 'generate_full_board' above and adds difficulty levels. There are 3 difficulty levels as of now 'easy', 'medium', 'hard'. This difficulty level is currently based on how many clues/numbers are visible at the initial stage. This needs to be updated after some further research.

In [468]:
import random

def generate_sudoku_game(difficulty_level, seed=None):
    if difficulty_level == "easy":
        num_clues = random.randint(36, 40)  # Adjust range as per desired difficulty
    elif difficulty_level == "medium":
        num_clues = random.randint(32, 35)
    elif difficulty_level == "hard":
        num_clues = random.randint(28, 31)
    else:
        raise ValueError("Invalid difficulty level")

    full_board = generate_full_board(seed)
    game_board = full_board.copy()

    # Randomly remove cells to create the game board
    cells_to_remove = 81 - num_clues
    for _ in range(cells_to_remove):
        row = random.randint(0, 8)
        col = random.randint(0, 8)
        while game_board[row * 9 + col] == 0:
            row = random.randint(0, 8)
            col = random.randint(0, 8)
        game_board[row * 9 + col] = 0

    return game_board


In [469]:
# Example usage:
game_board_easy = generate_sudoku_game("easy", seed=123)
print("Generated Sudoku Game:")
print_board(game_board_easy)


Generated Sudoku Game:
8 0 0 | 7 0 4 | 0 5 0 
0 3 0 | 9 5 1 | 0 2 0 
0 0 1 | 8 2 0 | 7 3 0 
---------------------
2 0 0 | 6 4 3 | 5 1 0 
6 0 3 | 5 1 0 | 0 0 7 
0 0 9 | 0 0 0 | 6 0 0 
---------------------
3 0 8 | 4 7 0 | 0 9 0 
0 7 0 | 0 0 5 | 0 6 0 
0 9 5 | 3 6 0 | 4 0 0 


In [470]:
# Example usage:
game_board = generate_sudoku_game("medium", seed=123)
print("Generated Sudoku Game:")
print_board(game_board)

Generated Sudoku Game:
8 0 0 | 7 0 0 | 0 5 0 
0 3 0 | 9 5 1 | 0 2 0 
0 0 1 | 8 2 0 | 0 3 0 
---------------------
2 0 0 | 6 4 3 | 5 1 0 
6 0 3 | 5 1 0 | 0 0 7 
0 0 9 | 0 0 0 | 0 0 0 
---------------------
3 0 8 | 4 7 0 | 0 9 0 
0 7 0 | 0 0 5 | 0 6 0 
0 9 5 | 3 0 0 | 0 0 0 


In [471]:
# Example usage:
game_board = generate_sudoku_game("hard", seed=123)
print("Generated Sudoku Game:")
print_board(game_board)

Generated Sudoku Game:
8 0 0 | 7 0 0 | 0 5 0 
0 3 0 | 9 0 1 | 0 2 0 
0 0 1 | 8 0 0 | 0 3 0 
---------------------
2 0 0 | 6 4 0 | 5 1 0 
6 0 3 | 5 1 0 | 0 0 7 
0 0 9 | 0 0 0 | 0 0 0 
---------------------
0 0 8 | 4 7 0 | 0 9 0 
0 7 0 | 0 0 5 | 0 6 0 
0 0 5 | 0 0 0 | 0 0 0 


Use the solver  

In [472]:
import time 
import importlib


In [473]:
import Solver_experiment_unified
importlib.reload(Solver_experiment_unified)


{'A1': '5', 'A2': '3', 'A3': '.', 'A4': '.', 'A5': '7', 'A6': '.', 'A7': '.', 'A8': '.', 'A9': '.', 'B1': '6', 'B2': '.', 'B3': '.', 'B4': '1', 'B5': '9', 'B6': '5', 'B7': '.', 'B8': '.', 'B9': '.', 'C1': '.', 'C2': '9', 'C3': '8', 'C4': '.', 'C5': '.', 'C6': '.', 'C7': '.', 'C8': '6', 'C9': '.', 'D1': '8', 'D2': '.', 'D3': '.', 'D4': '.', 'D5': '6', 'D6': '.', 'D7': '.', 'D8': '.', 'D9': '3', 'E1': '4', 'E2': '.', 'E3': '.', 'E4': '8', 'E5': '.', 'E6': '3', 'E7': '.', 'E8': '.', 'E9': '1', 'F1': '7', 'F2': '.', 'F3': '.', 'F4': '.', 'F5': '2', 'F6': '.', 'F7': '.', 'F8': '.', 'F9': '6', 'G1': '.', 'G2': '6', 'G3': '.', 'G4': '.', 'G5': '.', 'G6': '.', 'G7': '2', 'G8': '8', 'G9': '.', 'H1': '.', 'H2': '.', 'H3': '.', 'H4': '4', 'H5': '1', 'H6': '9', 'H7': '.', 'H8': '.', 'H9': '5', 'I1': '.', 'I2': '.', 'I3': '.', 'I4': '.', 'I5': '8', 'I6': '.', 'I7': '.', 'I8': '7', 'I9': '9'}
5 3 . | . 7 . | . . . 
6 . . | 1 9 5 | . . . 
. 9 8 | . . . | . 6 . 
---------------------
8 . . | . 6 . | .

<module 'Solver_experiment_unified' from 'c:\\Users\\luisf\\Desktop\\Semestre_4\\Data_structures\\project\\Data_structures\\Sudoku\\Sudoku\\flaskapp_sudoku\\flaskapp_sudoku\\app\\program\\Solver_experiment_unified.py'>

In [474]:
from Solver_experiment_unified import UnifiedSolver

In [475]:
game_board_easy = generate_sudoku_game("easy", seed=15)
print("Generated Sudoku Game:")
print_board(game_board_easy)

Generated Sudoku Game:
0 3 7 | 0 0 0 | 5 0 0 
2 0 6 | 5 1 4 | 9 0 7 
0 0 4 | 0 3 0 | 2 0 0 
---------------------
0 0 0 | 7 0 0 | 1 0 5 
7 0 0 | 1 4 5 | 3 0 0 
1 0 5 | 0 0 2 | 0 0 8 
---------------------
0 7 9 | 0 0 0 | 0 5 0 
6 0 0 | 4 0 0 | 0 0 9 
0 5 1 | 8 0 9 | 6 2 0 


In [477]:
game_board_easy_2d = convert_to_2d_list(game_board_easy)
solver_easy = UnifiedSolver(game_board_easy_2d)
print(type(game_board_easy_2d))  # Should show <class 'list'>
print(type(game_board_easy_2d[0]))

<class 'list'>
<class 'list'>


In [478]:
print(len(solver.board))

9


Tryout and solved

Partially solved
Note: the index error appears to be part of the board generator code line 60. 

In [482]:
if solver_easy.basic_solve():
    print("Sudoku Solved!")
    print("Solved Sudoku Game:")
    print_board(solver.board)
else:
    print("No solution exists for this Sudoku.")


Sudoku Solved!
Solved Sudoku Game:
[7, 9, 1, 3, 4, 8, 5, 6, 2] [3, 6, 8, 2, 5, 1, 9, 4, 7] [5, 2, 4, 9, 6, 7, 3, 1, 8] 
[6, 8, 3, 7, 2, 4, 1, 9, 5] [9, 4, 7, 8, 1, 5, 6, 2, 3] [2, 1, 5, 6, 9, 3, 7, 8, 4] 
[4, 3, 9, 5, 8, 6, 2, 7, 1] [8, 7, 6, 1, 3, 2, 4, 5, 9] [1, 5, 2, 4, 7, 9, 8, 3, 6] 


## Test advanced solver 

In [483]:
import time 
import importlib

In [484]:
import Solver_experiment_unified
importlib.reload(Solver_experiment_unified)
from Solver_experiment_unified import UnifiedSolver


{'A1': '5', 'A2': '3', 'A3': '.', 'A4': '.', 'A5': '7', 'A6': '.', 'A7': '.', 'A8': '.', 'A9': '.', 'B1': '6', 'B2': '.', 'B3': '.', 'B4': '1', 'B5': '9', 'B6': '5', 'B7': '.', 'B8': '.', 'B9': '.', 'C1': '.', 'C2': '9', 'C3': '8', 'C4': '.', 'C5': '.', 'C6': '.', 'C7': '.', 'C8': '6', 'C9': '.', 'D1': '8', 'D2': '.', 'D3': '.', 'D4': '.', 'D5': '6', 'D6': '.', 'D7': '.', 'D8': '.', 'D9': '3', 'E1': '4', 'E2': '.', 'E3': '.', 'E4': '8', 'E5': '.', 'E6': '3', 'E7': '.', 'E8': '.', 'E9': '1', 'F1': '7', 'F2': '.', 'F3': '.', 'F4': '.', 'F5': '2', 'F6': '.', 'F7': '.', 'F8': '.', 'F9': '6', 'G1': '.', 'G2': '6', 'G3': '.', 'G4': '.', 'G5': '.', 'G6': '.', 'G7': '2', 'G8': '8', 'G9': '.', 'H1': '.', 'H2': '.', 'H3': '.', 'H4': '4', 'H5': '1', 'H6': '9', 'H7': '.', 'H8': '.', 'H9': '5', 'I1': '.', 'I2': '.', 'I3': '.', 'I4': '.', 'I5': '8', 'I6': '.', 'I7': '.', 'I8': '7', 'I9': '9'}
5 3 . | . 7 . | . . . 
6 . . | 1 9 5 | . . . 
. 9 8 | . . . | . 6 . 
---------------------
8 . . | . 6 . | .

In [485]:
game_board_hard = generate_sudoku_game("hard", seed=200)
print("Generated Sudoku Game:")
print_board(game_board_hard)


Generated Sudoku Game:
0 0 0 | 0 0 0 | 0 0 1 
0 9 0 | 0 0 1 | 5 0 3 
0 4 0 | 0 8 3 | 2 0 0 
---------------------
0 0 0 | 3 0 9 | 4 0 0 
0 0 9 | 0 1 0 | 0 0 2 
0 0 6 | 8 0 2 | 3 7 9 
---------------------
9 0 5 | 0 2 8 | 1 0 4 
0 0 0 | 0 0 0 | 0 0 0 
0 0 4 | 0 3 0 | 7 2 0 


In [486]:
game_board_hard_2d = convert_to_2d_list(game_board_hard)
solver_hard = UnifiedSolver(game_board_hard_2d, difficulty='hard')

print(type(game_board_hard_2d))  # Should show <class 'list'>
print(type(game_board_hard_2d[0]))
print(game_board_hard_2d)


<class 'list'>
<class 'list'>
[[0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 9, 0, 0, 0, 1, 5, 0, 3], [0, 4, 0, 0, 8, 3, 2, 0, 0], [0, 0, 0, 3, 0, 9, 4, 0, 0], [0, 0, 9, 0, 1, 0, 0, 0, 2], [0, 0, 6, 8, 0, 2, 3, 7, 9], [9, 0, 5, 0, 2, 8, 1, 0, 4], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 4, 0, 3, 0, 7, 2, 0]]


In [487]:
if solver_hard.solve():
    print("Sudoku Solved!")
    print("Solved Sudoku Game:")
    solved_flat_board = [cell for row in solver_hard.board for cell in row]
    print_board(solved_flat_board)
else:
    print("No solution exists for this Sudoku.")

ValueError: Expected integer for num, got <class 'set'> with value {2}